In [ ]:
# default_exp util

# Util ssb

> API details.

In [ ]:
#export 
from smpr3d.torch_imports import *
from smpr3d.util import fftshift_checkerboard, Param
import numpy as np

# def zernike_aberrations(qx, qy, lam, C):
#     """
#     Calculates the geometrical aberrations up to the order specified in C. C is a complex array specifying the
#     aberration coefficients.
# 
#     :param qx: x coordinate system, can be (i_batch, qxx, qxy)
#     :param qy: y coordinate system, can be (i_batch, qyx, qyy)
#     :param lam: wavelength in A
#     :param C: (n_batch, m_max, n_max) complex array
#     :return: chi, the real aberration function
#     """
#     if isinstance(qx, np.ndarray):
#         xp = np
# 
#         if C.ndim == 3:
#             n_batch, m_max, n_max = C.shape
#         elif C.ndim == 2:
#             m_max, n_max = C.shape
#             n_batch = 1
#             C = C[None, ...]
#         else:
#             raise RuntimeError("Aberration coefficient array has wrong dimension")
# 
#         sh = (n_batch, *qx.shape)
#         chi = xp.zeros(sh, dtype=np.float32) + 1j * xp.zeros(sh, dtype=np.float32)
#         ω = lam * (qx + 1j * qy)
# 
#         for i_batch in range(n_batch):
#             for m in range(m_max):
#                 if m <= 3:
#                     n_max = m + 1
#                 elif m <= 5:
#                     n_max = m // 2
#                 else:
#                     n_max = 1
#                 for n in range(n_max):
#                     if m > 0 or n > 0:
#                         # print(f"C[{m}, {n}] = {C[i_batch, m, n]}")
#                         ab = C[i_batch, m, n] * (ω ** m) * (ω.conj() ** n)
#                         ab /= (m + n)
#                         chi[i_batch] += ab
#         chi = 2 * np.pi / lam * chi.real
#         chi = xp.squeeze(chi)
#     elif isinstance(qx, th.Tensor):
#         if C.ndimension() == 4:
#             n_batch, m_max, n_max, _ = C.shape
#         elif C.ndimension() == 3:
#             m_max, n_max, _ = C.shape
#             n_batch = 1
#             C = C.unsqueeze(0)
#         else:
#             raise RuntimeError("Aberration coefficient array has wrong dimension")
# 
#         sh = (n_batch, *qx.shape)
# 
#         chi = th.zeros(sh, dtype=th.float32)
# 
#         # print(chi.shape)
# 
#         ω = th.zeros_like(qx)
#         ω[re] = lam * qx
#         ω[im] = lam * qy
#         C_mn = make_real(th.zeros(qx.shape))
#         for i_batch in range(n_batch):
#             for m in range(m_max):
#                 if m <= 3:
#                     n_max = m + 1
#                 elif m <= 5:
#                     n_max = m // 2
#                 else:
#                     n_max = 1
#                 for n in range(n_max):
#                     if m > 0 or n > 0:
#                         # print(f"C[{m}, {n}] = {C[i_batch, m, n]}")
#                         C_mn[re] = C[i_batch, m, n][re]
#                         C_mn[im] = C[i_batch, m, n][im]
#                         ab = complex_mul(C_mn, cpow(ω, n))
#                         ab = complex_mul(ab, cpow(conj(ω), n))
#                         ab /= (m + n)
#                         # print(chi.shape)
#                         chi[i_batch] += ab
#         chi = 2 * np.pi / lam * chi[re]
#         chi = chi.squeeze()
#     return chi

def cartesian_aberrations_single(qx, qy, lam, C):
    """
    Zernike polynomials in the cartesian coordinate system
    :param qx:
    :param qy:
    :param lam: wavelength in Angstrom
    :param C:   (12 ,)
    :return:
    """

    u = qx * lam
    v = qy * lam
    u2 = u ** 2
    u3 = u ** 3
    u4 = u ** 4
    # u5 = u ** 5

    v2 = v ** 2
    v3 = v ** 3
    v4 = v ** 4
    # v5 = v ** 5

    aberr = Param()
    aberr.C1 = C[0]
    aberr.C12a = C[1]
    aberr.C12b = C[2]
    aberr.C21a = C[3]
    aberr.C21b = C[4]
    aberr.C23a = C[5]
    aberr.C23b = C[6]
    aberr.C3 = C[7]
    aberr.C32a = C[8]
    aberr.C32b = C[9]
    aberr.C34a = C[10]
    aberr.C34b = C[11]

    chi = 0

    # r-2 = x-2 +y-2.
    chi += 1 / 2 * aberr.C1 * (u2 + v2) # r^2
    #r-2 cos(2*phi) = x"2 -y-2.
    # r-2 sin(2*phi) = 2*x*y.
    chi += 1 / 2 * (aberr.C12a * (u2 - v2) + 2 * aberr.C12b * u * v) # r^2 cos(2 phi) + r^2 sin(2 phi)
    # r-3 cos(3*phi) = x-3 -3*x*y'2. r"3 sin(3*phi) = 3*y*x-2 -y-3.
    chi += 1 / 3 * (aberr.C23a * (u3 - 3 * u * v2) + aberr.C23b * (3 * u2 * v - v3))# r^3 cos(3phi) + r^3 sin(3 phi)
    # r-3 cos(phi) = x-3 +x*y-2.
    # r-3 sin(phi) = y*x-2 +y-3.
    chi += 1 / 3 * (aberr.C21a * (u3 + u * v2) + aberr.C21b * (v3 + u2 * v))# r^3 cos(phi) + r^3 sin(phi)
    # r-4 = x-4 +2*x-2*y-2 +y-4.
    chi += 1 / 4 * aberr.C3 * (u4 + v4 + 2 * u2 * v2)# r^4
    # r-4 cos(4*phi) = x-4 -6*x-2*y-2 +y-4.
    chi += 1 / 4 * aberr.C34a * (u4 - 6 * u2 * v2 + v4)# r^4 cos(4 phi)
    # r-4 sin(4*phi) = 4*x-3*y -4*x*y-3.
    chi += 1 / 4 * aberr.C34b * (4 * u3 * v - 4 * u * v3) # r^4 sin(4 phi)
    # r-4 cos(2*phi) = x-4 -y-4.
    chi += 1 / 4 * aberr.C32a * (u4 - v4)
    # r-4 sin(2*phi) = 2*x-3*y +2*x*y-3.
    chi += 1 / 4 * aberr.C32b * (2 * u3 * v + 2 * u * v3)
    # r-5 cos(phi) = x-5 +2*x-3*y-2 +x*y-4.
    # r-5 sin(phi) = y*x"4 +2*x-2*y-3 +y-5.
    # r-5 cos(3*phi) = x-5 -2*x-3*y-2 -3*x*y-4.
    # r-5 sin(3*phi) = 3*y*x-4 +2*x-2*y-3 -y-5.
    # r-5 cos(5*phi) = x-5 -10*x-3*y-2 +5*x*y-4.
    # r-5 sin(5*phi) = 5*y*x-4 -10*x-2*y-3 +y-5.

    chi *= 2 * np.pi / lam

    return chi

def cartesian_aberrations(qx, qy, lam, C):
    """
    Zernike polynomials in the cartesian coordinate system
    :param qx:
    :param qy:
    :param lam: wavelength in Angstrom
    :param C:   12 x D
    :return:
    """

    u = qx * lam
    v = qy * lam
    u2 = u ** 2
    u3 = u ** 3
    u4 = u ** 4
    # u5 = u ** 5

    v2 = v ** 2
    v3 = v ** 3
    v4 = v ** 4
    # v5 = v ** 5

    aberr = Param()
    aberr.C1 = C[0].unsqueeze(1).unsqueeze(1)
    aberr.C12a = C[1].unsqueeze(1).unsqueeze(1)
    aberr.C12b = C[2].unsqueeze(1).unsqueeze(1)
    aberr.C21a = C[3].unsqueeze(1).unsqueeze(1)
    aberr.C21b = C[4].unsqueeze(1).unsqueeze(1)
    aberr.C23a = C[5].unsqueeze(1).unsqueeze(1)
    aberr.C23b = C[6].unsqueeze(1).unsqueeze(1)
    aberr.C3 = C[7].unsqueeze(1).unsqueeze(1)
    aberr.C32a = C[8].unsqueeze(1).unsqueeze(1)
    aberr.C32b = C[9].unsqueeze(1).unsqueeze(1)
    aberr.C34a = C[10].unsqueeze(1).unsqueeze(1)
    aberr.C34b = C[11].unsqueeze(1).unsqueeze(1)

    chi = 0

    # r-2 = x-2 +y-2.
    chi += 1 / 2 * aberr.C1 * (u2 + v2) # r^2
    #r-2 cos(2*phi) = x"2 -y-2.
    # r-2 sin(2*phi) = 2*x*y.
    chi += 1 / 2 * (aberr.C12a * (u2 - v2) + 2 * aberr.C12b * u * v) # r^2 cos(2 phi) + r^2 sin(2 phi)
    # r-3 cos(3*phi) = x-3 -3*x*y'2. r"3 sin(3*phi) = 3*y*x-2 -y-3.
    chi += 1 / 3 * (aberr.C23a * (u3 - 3 * u * v2) + aberr.C23b * (3 * u2 * v - v3))# r^3 cos(3phi) + r^3 sin(3 phi)
    # r-3 cos(phi) = x-3 +x*y-2.
    # r-3 sin(phi) = y*x-2 +y-3.
    chi += 1 / 3 * (aberr.C21a * (u3 + u * v2) + aberr.C21b * (v3 + u2 * v))# r^3 cos(phi) + r^3 sin(phi)
    # r-4 = x-4 +2*x-2*y-2 +y-4.
    chi += 1 / 4 * aberr.C3 * (u4 + v4 + 2 * u2 * v2)# r^4
    # r-4 cos(4*phi) = x-4 -6*x-2*y-2 +y-4.
    chi += 1 / 4 * aberr.C34a * (u4 - 6 * u2 * v2 + v4)# r^4 cos(4 phi)
    # r-4 sin(4*phi) = 4*x-3*y -4*x*y-3.
    chi += 1 / 4 * aberr.C34b * (4 * u3 * v - 4 * u * v3) # r^4 sin(4 phi)
    # r-4 cos(2*phi) = x-4 -y-4.
    chi += 1 / 4 * aberr.C32a * (u4 - v4)
    # r-4 sin(2*phi) = 2*x-3*y +2*x*y-3.
    chi += 1 / 4 * aberr.C32b * (2 * u3 * v + 2 * u * v3)
    # r-5 cos(phi) = x-5 +2*x-3*y-2 +x*y-4.
    # r-5 sin(phi) = y*x"4 +2*x-2*y-3 +y-5.
    # r-5 cos(3*phi) = x-5 -2*x-3*y-2 -3*x*y-4.
    # r-5 sin(3*phi) = 3*y*x-4 +2*x-2*y-3 -y-5.
    # r-5 cos(5*phi) = x-5 -10*x-3*y-2 +5*x*y-4.
    # r-5 sin(5*phi) = 5*y*x-4 -10*x-2*y-3 +y-5.

    chi *= 2 * np.pi / lam

    return chi


def aperture(q: th.Tensor, lam, alpha_max, edge=2):
    ktheta = th.asin(q.norm(dim=0) * lam)
    qmax = alpha_max / lam
    dk = q[0][1][0]

    arr = th.zeros_like(q[1])
    arr[ktheta < alpha_max] = 1
    if edge > 0:
        dEdge = edge / (qmax / dk)  # fraction of aperture radius that will be smoothed
        # some fancy indexing: pull out array elements that are within
        #    our smoothing edges
        ind = (ktheta / alpha_max > (1 - dEdge)) * (ktheta / alpha_max < (1 + dEdge))
        arr[ind] = 0.5 * (1 - th.sin(np.pi / (2 * dEdge) * (ktheta[ind] / alpha_max - 1)))
    return arr


class ZernikeProbe2(nn.Module):
    def __init__(self, q: th.Tensor, lam, fft_shifted=True):
        """
        Creates an aberration surface from aberration coefficients. The output is backpropable

        :param q: 2 x M1 x M2 tensor of x coefficients of reciprocal space
        :param lam: wavelength in Angstrom
        :param C: aberration coefficients
        :return: (maximum size of all aberration tensors) x MY x MX
        """

        super(ZernikeProbe2, self).__init__()
        self.q = q
        self.lam = lam
        self.fft_shifted = fft_shifted

        if self.fft_shifted:
            cb = fftshift_checkerboard(self.q.shape[1] // 2, self.q.shape[2] // 2)
            self.cb = th.from_numpy(cb).float().to(q.device)

    def forward(self, C, A):
        chi = cartesian_aberrations(self.q[1], self.q[0], self.lam, C)
        Psi = th.exp(-1j*chi) * A.expand_as(chi)

        if self.fft_shifted:
            Psi = Psi * self.cb

        return Psi

class ZernikeProbeSingle(nn.Module):
    def __init__(self, q: th.Tensor, lam, fft_shifted=True):
        """
        Creates an aberration surface from aberration coefficients. The output is backpropable

        :param q: 2 x M1 x M2 tensor of x coefficients of reciprocal space
        :param lam: wavelength in Angstrom
        :param C: aberration coefficients
        :return: (maximum size of all aberration tensors) x MY x MX
        """

        super(ZernikeProbeSingle, self).__init__()
        self.q = q
        self.lam = lam
        self.fft_shifted = fft_shifted

        if self.fft_shifted:
            cb = fftshift_checkerboard(self.q.shape[1] // 2, self.q.shape[2] // 2)
            self.cb = th.from_numpy(cb).float().to(q.device)

    def forward(self, C, A):
        chi = cartesian_aberrations_single(self.q[1], self.q[0], self.lam, C)
        Psi = th.exp(-1j*chi) * A.expand_as(chi)

        if self.fft_shifted:
            Psi = Psi * self.cb

        return Psi

class ZernikeProbe(nn.Module):
    def __init__(self, q: th.Tensor, lam, A_init, A_requires_grad, fft_shifted=True,
                 C1=th.zeros(1), C12a=th.zeros(1), C12b=th.zeros(1), C21a=th.zeros(1), C21b=th.zeros(1),
                 C23a=th.zeros(1), C23b=th.zeros(1), C3=th.zeros(1), C32a=th.zeros(1), C32b=th.zeros(1),
                 C34a=th.zeros(1), C34b=th.zeros(1)):
        """
        Creates an aberration surface from aberration coefficients. The output is backpropable

        :param qx: M1 x M2 tensor of x coefficients of reciprocal space
        :param qy: M1 x M2 tensor of y coefficients of reciprocal space
        :param lam: wavelength in Angstrom
        :param C: shift                               in Angstrom
        :return: (maximum size of all aberration tensors) x MY x MX
        """

        super(ZernikeProbe, self).__init__()
        self.qx = q[1]
        self.qy = q[0]
        self.lam = lam
        self.fft_shifted = fft_shifted
        self.dtype = q.dtype
        d = self.dtype
        # defocus
        self.C1 = nn.Parameter(data=C1.type(d), requires_grad=th.max(th.abs(C1)).item() > 1e-10)
        # two-fold stig
        self.C12a = nn.Parameter(data=C12a.type(d), requires_grad=th.max(th.abs(C12a)).item() > 1e-10).to(C1.device)
        self.C12b = nn.Parameter(data=C12b.type(d), requires_grad=th.max(th.abs(C12b)).item() > 1e-10).to(C1.device)

        # axial coma
        self.C21a = nn.Parameter(data=C21a.type(d), requires_grad=th.max(th.abs(C21a)).item() > 1e-10).to(C1.device)
        self.C21b = nn.Parameter(data=C21b.type(d), requires_grad=th.max(th.abs(C21b)).item() > 1e-10).to(C1.device)
        # 3-fold stig
        self.C23a = nn.Parameter(data=C23a.type(d), requires_grad=th.max(th.abs(C23a)).item() > 1e-10).to(C1.device)
        self.C23b = nn.Parameter(data=C23b.type(d), requires_grad=th.max(th.abs(C23b)).item() > 1e-10).to(C1.device)
        # spherical
        self.C3 = nn.Parameter(data=C3.type(d), requires_grad=th.max(th.abs(C3)).item() > 1e-10).to(C1.device)
        # star
        self.C32a = nn.Parameter(data=C32a.type(d), requires_grad=th.max(th.abs(C32a)).item() > 1e-10).to(C1.device)
        self.C32b = nn.Parameter(data=C32b.type(d), requires_grad=th.max(th.abs(C32b)).item() > 1e-10).to(C1.device)
        # 4-fold stig
        self.C34a = nn.Parameter(data=C34a.type(d), requires_grad=th.max(th.abs(C34a)).item() > 1e-10).to(C1.device)
        self.C34b = nn.Parameter(data=C34b.type(d), requires_grad=th.max(th.abs(C34b)).item() > 1e-10).to(C1.device)

        self.A = nn.Parameter(data=A_init.type(d), requires_grad=A_requires_grad).to(C1.device)

        self.optimized_parameters_ = []

        if self.fft_shifted:
            self.cb = th.from_numpy(fftshift_checkerboard(self.qx.shape[0] // 2, self.qx.shape[1] // 2)).float().to(
                q.device).type(d)

        for p in self.parameters():
            if p.requires_grad:
                self.optimized_parameters_.append(p)

    def optimized_parameters(self):
        return self.optimized_parameters_

    def forward(self):
        max_size = np.max(
            [a.shape[0] if len(a.shape) > 0 else 1 for a in
             [self.C1, self.C12a, self.C12b, self.C21a, self.C21b, self.C23a, self.C23b, self.C3,
              self.C32a, self.C32b, self.C34a, self.C34b]])
        C = th.stack((self.C1.expand(max_size), self.C12a.expand(max_size), self.C12b.expand(max_size),
                      self.C21a.expand(max_size), self.C21b.expand(max_size), self.C23a.expand(max_size),
                      self.C23b.expand(max_size), self.C3.expand(max_size), self.C32a.expand(max_size),
                      self.C32b.expand(max_size), self.C34a.expand(max_size), self.C34b.expand(max_size)), 0)

        chi = cartesian_aberrations(self.qx, self.qy, self.lam, C).type(self.dtype)
        Psi = th.exp(-1j*chi) * A.expand_as(chi)

        if self.fft_shifted:
            Psi = Psi * self.cb

        return Psi

